In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data=pd.read_csv("C:/Users/ritvi/east_nondomestic.csv")

In [ ]:
data['date']=pd.to_datetime(data['date'])
data.set_index('date',inplace=True)
data.head()

In [ ]:
    import plotly.offline as py
    py.init_notebook_mode(connected=True)
    py.enable_mpl_offline()

    import cufflinks as cf
    cf.go_offline()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data, model='multiplicative')

fig = result.plot()

In [ ]:
py.iplot_mpl(fig)

In [ ]:
from pmdarima import auto_arima

In [ ]:
stepwise_model = auto_arima(data, start_p=0, start_q=0,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

In [ ]:
print(stepwise_model.aic())

In [ ]:
# For the Test: we'll need to chop off a portion of our latest data
test = data.loc['2023-05-15':]

# Fore the Train: we'll train on the rest of the data after split the test portion
train = data.loc['2023-01-15':'2023-05-14']

In [ ]:
stepwise_model.fit(train)

In [ ]:
future_forecast = stepwise_model.predict(n_periods=42)
print(future_forecast)

In [ ]:
future_forecast = pd.DataFrame(future_forecast, index=test.index, columns=['Prediction'])

In [ ]:
pd.concat([test, future_forecast], axis=1).iplot()

In [ ]:
pd.concat([data,future_forecast],axis=1).iplot()

In [ ]:
stepwise_model.fit(data)

In [ ]:
future_forecast_1year = stepwise_model.predict(n_periods=42)
val=future_forecast_1year

In [ ]:
next_year = [pd.to_datetime('2023-06-24'),
            pd.to_datetime('2023-06-26'),
            pd.to_datetime('2023-06-27'),
            pd.to_datetime('2023-06-28'),
            pd.to_datetime('2023-06-29'),
            pd.to_datetime('2023-06-30'),
            pd.to_datetime('2023-07-01'),
            pd.to_datetime('2023-07-02'),
            pd.to_datetime('2023-07-03'),
            pd.to_datetime('2023-07-04'),
            pd.to_datetime('2023-07-05'),
            pd.to_datetime('2023-07-06'),
            pd.to_datetime('2023-07-07'),]

In [ ]:
future_forecast_1year = pd.DataFrame(future_forecast_1year, index=next_year, columns=['Prediction'])

In [ ]:
pd.concat([data,future_forecast_1year],axis=1).iplot()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
custom_dates = pd.date_range(start='2023-05-05', periods=13, freq='D')  

In [ ]:
metric = stepwise_model.predict(n_periods=13, index=custom_dates)

In [ ]:
#mae = mean_absolute_error(test, val)
#rmse = np.sqrt(mean_squared_error(test, val))
mape = mean_absolute_percentage_error(test, val)

In [ ]:
print("MAPE:", mape)